### merge_ordered()
default = outer merge

In [86]:
import pandas as pd

austin = pd.read_csv('austin.csv', delim_whitespace=True)
houston = pd.read_csv('houston.csv', delim_whitespace=True)

In [89]:
# Perform the first ordered merge: tx_weather
tx_weather = pd.merge_ordered(austin,houston)
tx_weather

,date,ratings
0,2016-01-01,Cloudy
1,2016-01-04,Rainy
2,2016-01-17,Sunny
3,2016-02-08,Cloudy
4,2016-03-01,Sunny


All information about the city is lost

In [90]:
# Perform the second ordered merge: tx_weather_suff
tx_weather_suff = pd.merge_ordered(austin,houston, on ='date', suffixes=['_aus', '_hus'])
tx_weather_suff

,date,ratings_aus,ratings_hus
0,2016-01-01,Cloudy,Cloudy
1,2016-01-04,NaN,Rainy
2,2016-01-17,Sunny,NaN
3,2016-02-08,Cloudy,NaN
4,2016-03-01,NaN,Sunny


In [91]:
# Perform the third ordered merge: tx_weather_ffill
tx_weather_ffill = pd.merge_ordered(austin,houston, on='date', suffixes=['_aus', '_hus'], fill_method='ffill')
tx_weather_ffill

,date,ratings_aus,ratings_hus
0,2016-01-01,Cloudy,Cloudy
1,2016-01-04,Cloudy,Rainy
2,2016-01-17,Sunny,Rainy
3,2016-02-08,Cloudy,Rainy
4,2016-03-01,Cloudy,Sunny


###  merge_asof()
Similar to `pd.merge_ordered()`, the `pd.merge_asof()` function will also merge values in order using the on column, but for each row in the left DataFrame, **only rows from the right DataFrame whose 'on' column values are less than the left value will be kept**.

In [104]:
import pandas as pd

auto = pd.read_csv('automobiles.csv', parse_dates=['yr'])
oil = pd.read_csv('oil_price.csv', parse_dates=['Date'])

In [105]:
auto.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,1970-01-01,US,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,1970-01-01,US,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,1970-01-01,US,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,1970-01-01,US,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,1970-01-01,US,ford torino


In [106]:
oil.head()

,Date,Price
0,1970-01-01,3.35
1,1970-02-01,3.35
2,1970-03-01,3.35
3,1970-04-01,3.35
4,1970-05-01,3.35


In [109]:
# Merge auto and oil: merged
merged = pd.merge_asof(auto, oil, left_on='yr', right_on='Date')
merged.tail()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,Date,Price
387,27.0,4,140.0,86,2790,15.6,1982-01-01,US,ford mustang gl,1982-01-01,33.85
388,44.0,4,97.0,52,2130,24.6,1982-01-01,Europe,vw pickup,1982-01-01,33.85
389,32.0,4,135.0,84,2295,11.6,1982-01-01,US,dodge rampage,1982-01-01,33.85
390,28.0,4,120.0,79,2625,18.6,1982-01-01,US,ford ranger,1982-01-01,33.85
391,31.0,4,119.0,82,2720,19.4,1982-01-01,US,chevy s-10,1982-01-01,33.85


In [111]:
# Resample merged: yearly
yearly = merged.resample('A', on='Date')[['mpg','Price']].mean()
yearly

,mpg,Price
Date,,
1970-12-31,17.689655,3.35
1971-12-31,21.111111,3.56
1972-12-31,18.714286,3.56
1973-12-31,17.100000,3.56
1974-12-31,22.769231,10.11
1975-12-31,20.266667,11.16
1976-12-31,21.573529,11.16
1977-12-31,23.375000,13.90
1978-12-31,24.061111,14.85


In [112]:
# Pearson correlation between the resampled 'Price' and 'mpg'
yearly.corr()

,mpg,Price
mpg,1.000000,0.948677
Price,0.948677,1.000000


It looks like there is a strong correlation between miles per gallon and the price of oil!!